<a href="https://colab.research.google.com/github/GuptaNavdeep1983/CS688/blob/main/Assignment_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References:

https://datascience.stackexchange.com/questions/54904/how-to-avoid-tokenizing-w-sklearn-feature-extraction


https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

https://www.freecodecamp.org/news/an-introduction-to-bag-of-words-and-how-to-code-it-in-python-for-nlp-282e87a9da04/

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer.transform

https://radimrehurek.com/gensim/auto_examples/core/run_corpora_and_vector_spaces.html

In [ ]:
pip install gensim

In [310]:
import statsmodels.api as sm 
import pylab as py 
import matplotlib.pyplot as plt
import pandas as pd
import urllib.request
import datetime
from zipfile import ZipFile
from gzip import decompress
from json import loads
from requests import get
import requests, zipfile, io
from bs4 import BeautifulSoup
import math
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import re
import gensim
from gensim import corpora
from gensim.models import LsiModel, LdaModel
from gensim.models.coherencemodel import CoherenceModel
from gensim.test.utils import common_texts
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation, strip_punctuation2, strip_short, strip_numeric, stem_text
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

In [312]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [305]:
def get_crowd_funding_data_links(filter_by_years):
    # get the content of the website
    content = urllib.request.urlopen('https://webrobots.io/indiegogo-dataset/').read()
    soup = BeautifulSoup(content,features='html.parser')
    
    all_stories = pd.DataFrame([], columns=['Year', 'Month', 'json', 'csv'])
    parent_div = soup.find(name='div', attrs={'class':'fusion-text'})

    # Iterate through the divs to find the list of Month-Year wise anchor tags
    index = 0
    for year_data in parent_div.find_all(name='ul'):
        all_months = year_data.find_all('li')
        for month in all_months:
            complete_text = month.text
            date_time_obj = datetime.datetime.strptime((complete_text.split('[')[0]).strip(), '%Y-%m-%d')
            if date_time_obj.year in filter_by_years:
                all_stories.loc[index, "Year"] = date_time_obj.year
                all_stories.loc[index, "Month"] = str(date_time_obj.month) if date_time_obj.month >=10 else f'0{date_time_obj.month}'
                all_stories.loc[index, "json"] = month.find_all('a')[0]["href"]
                all_stories.loc[index, "csv"] = month.find_all('a')[1]["href"]
                index = index + 1
    return all_stories

def get_current_index_data(df, indx, columns):
    r = requests.get(df["csv"][indx])
    z = zipfile.ZipFile(io.BytesIO(r.content))
    csv_data = z.open("Indiegogo.csv")
    data_df = pd.read_csv(csv_data, usecols=columns, dtype={'title':str, 'tagline':str})
    return data_df
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

In [313]:
df = get_crowd_funding_data_links([2020])
stemmer = SnowballStemmer("english")
documents = []
for indx in df.head(5).index:
    current_month_data = get_current_index_data(df, indx, ["title", "tagline"])
    current_month_data.dropna(axis=0, inplace=True)
    for index, row in current_month_data.iterrows():
      # all_content = strip_numeric(strip_short(strip_punctuation2(strip_punctuation(remove_stopwords(row['tagline']))), minsize=4))
      all_content = preprocess(row['tagline'])
      documents.append(all_content)

In [316]:
# print(common_texts)
common_dictionary = corpora.Dictionary(documents)
common_dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n= 100000)

In [167]:
df = pd.DataFrame(documents, columns=['content'])
vectorizer = CountVectorizer(input='content',stop_words='english', analyzer='word', token_pattern=r'\b[a-zA-Z]{2,}\b',max_features=100)
doc_term_matrix = vectorizer.fit_transform(df['content'])
# doc_term_matrix = vectorizer.transform(df['content'])
feature_names = vectorizer.get_feature_names()
df_dictionary = pd.DataFrame(feature_names)
dictionary = df_dictionary.to_dict()[0]
# df_doc_term_matrix = pd.DataFrame(doc_term_matrix.toarray())
term_doc_matrix = doc_term_matrix.transpose()

In [317]:

common_corpus = [common_dictionary.doc2bow(doc) for doc in documents]
print(len(common_corpus))

164361


In [318]:
def create_gensim_lsi_model(common_corpus, common_dictionary):
    # generate LSI model
    lsamodel = LsiModel(common_corpus, id2word = common_dictionary, num_topics=5)  # train model
    print(lsamodel.print_topics())
    return lsamodel

In [320]:
def create_gensim_lda_model(common_corpus, common_dictionary):
    # generate LDA model
    lsamodel = gensim.models.LdaMulticore(common_corpus, num_topics=5, id2word=common_dictionary, passes=10, workers=2)  # train model
    print(lsamodel.print_topics())
    return lsamodel

In [299]:
lsi_model = create_gensim_lsi_model(common_corpus=common_corpus, common_dictionary=common_dictionary)


[(0, '0.927*"The" + 0.225*"Help" + 0.113*"new" + 0.101*"world" + 0.078*"help" + 0.046*"World" + 0.044*"need" + 0.040*"game" + 0.038*"book" + 0.035*"First"'), (1, '0.891*"Help" + -0.299*"The" + 0.180*"new" + 0.112*"help" + 0.085*"fund" + 0.074*"bring" + 0.056*"raise" + 0.053*"create" + 0.052*"need" + 0.050*"build"'), (2, '0.682*"help" + 0.504*"new" + -0.311*"Help" + 0.255*"need" + -0.126*"The" + 0.097*"world" + 0.076*"Please" + 0.067*"needs" + 0.066*"book" + 0.063*"people"'), (3, '-0.785*"new" + 0.551*"help" + 0.196*"need" + 0.081*"Help" + 0.057*"Please" + -0.054*"game" + -0.044*"way" + 0.038*"people" + -0.038*"album" + -0.034*"series"'), (4, '0.894*"world" + 0.186*"game" + 0.155*"book" + -0.151*"new" + -0.147*"help" + -0.106*"The" + 0.095*"people" + -0.065*"Help" + -0.049*"need" + 0.049*"way"')]


In [321]:
lda_model = create_gensim_lda_model(common_corpus, common_dictionary)

[(0, '0.025*"support" + 0.023*"fund" + 0.017*"rais" + 0.015*"communiti" + 0.013*"need" + 0.012*"love" + 0.011*"film" + 0.010*"album" + 0.009*"music" + 0.008*"artist"'), (1, '0.021*"power" + 0.012*"design" + 0.012*"smart" + 0.011*"light" + 0.010*"devic" + 0.010*"bike" + 0.009*"world" + 0.009*"charg" + 0.009*"portabl" + 0.009*"build"'), (2, '0.021*"book" + 0.014*"stori" + 0.014*"life" + 0.012*"world" + 0.011*"bring" + 0.010*"live" + 0.010*"photographi" + 0.009*"chang" + 0.008*"seri" + 0.007*"comic"'), (3, '0.012*"camera" + 0.011*"world" + 0.010*"creat" + 0.009*"better" + 0.008*"organ" + 0.008*"food" + 0.008*"free" + 0.008*"time" + 0.008*"design" + 0.007*"need"'), (4, '0.047*"game" + 0.019*"play" + 0.014*"world" + 0.012*"card" + 0.010*"adventur" + 0.007*"board" + 0.006*"action" + 0.006*"drive" + 0.006*"player" + 0.006*"design"')]


In [ ]:
lsi_model.show_topics()


In [ ]:
topics = lda_model.show_topics()
for topic in topics:
  terms = lda_model.get_topic_terms(topic[0])
  print(f"Topic:{topic[0]} Terms,")
  for term in terms:
    print(common_dictionary[term[0]])
# for tuple in out:

# lda_model.print_topics()
# lda_model.top_topics(corpus=common_corpus, dictionary=common_dictionary)

In [324]:
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.025*"support" + 0.023*"fund" + 0.017*"rais" + 0.015*"communiti" + 0.013*"need" + 0.012*"love" + 0.011*"film" + 0.010*"album" + 0.009*"music" + 0.008*"artist"


Topic: 1 
Words: 0.021*"power" + 0.012*"design" + 0.012*"smart" + 0.011*"light" + 0.010*"devic" + 0.010*"bike" + 0.009*"world" + 0.009*"charg" + 0.009*"portabl" + 0.009*"build"


Topic: 2 
Words: 0.021*"book" + 0.014*"stori" + 0.014*"life" + 0.012*"world" + 0.011*"bring" + 0.010*"live" + 0.010*"photographi" + 0.009*"chang" + 0.008*"seri" + 0.007*"comic"


Topic: 3 
Words: 0.012*"camera" + 0.011*"world" + 0.010*"creat" + 0.009*"better" + 0.008*"organ" + 0.008*"food" + 0.008*"free" + 0.008*"time" + 0.008*"design" + 0.007*"need"


Topic: 4 
Words: 0.047*"game" + 0.019*"play" + 0.014*"world" + 0.012*"card" + 0.010*"adventur" + 0.007*"board" + 0.006*"action" + 0.006*"drive" + 0.006*"player" + 0.006*"design"




In [ ]:
from scipy.cluster import hierarchy